In [1]:
import transformers
import torch

model = transformers.BertForMaskedLM.from_pretrained("TurkuNLP/bert-base-finnish-cased-v1") 
model.eval()
if torch.cuda.is_available():
    model = model.cuda()
tokenizer = transformers.BertTokenizer.from_pretrained("TurkuNLP/bert-base-finnish-cased-v1")

#Use "bert-base-finnish-uncased-v1" if you want to use the uncased model.

Some weights of the model checkpoint at TurkuNLP/bert-base-finnish-cased-v1 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
#from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("TurkuNLP/bert-base-finnish-cased-v1")
model = AutoModelForMaskedLM.from_pretrained("TurkuNLP/bert-base-finnish-cased-v1")

#classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
#generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

Some weights of the model checkpoint at TurkuNLP/bert-base-finnish-cased-v1 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
The model 'BertForMaskedLM' is not supported for sentiment-analysis. Supported models are ['PerceiverForSequenceClassification', 'QDQBertForSequenceClassification', 'FNetForSequenceClassification', 'GPTJForSequenceClassification', 'LayoutLMv2ForSequenceClassification', 'RemBertForSequenceClassification', 'CanineForSequenceClassification', 'RoForme

In [35]:
#classifier("Suomenruotsalaiset ovat täällä")
#https://huggingface.co/course/chapter2/2?fw=pt
#https://huggingface.co/TurkuNLP/bert-base-finnish-cased-v1
#raw_inputs = [
#    "Tämä on hauskaa.",
#    "Minä vihaan tätä!",
#]
raw_inputs = "Tämä on hauskaa"
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)
outputs = model(**inputs)
print(outputs.logits.shape)
# logits, the raw, unnormalized scores outputted by the last layer of the model. 
#To be converted to probabilities, they need to go through a SoftMax layer 

{'input_ids': tensor([[ 102, 1131,  145, 6183,  103]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}
torch.Size([1, 5, 50105])


In [3]:
from transformers import pipeline

generator = pipeline("text-generation")
generator("In this course, we will teach you how to")

No model was supplied, defaulted to gpt2 (https://huggingface.co/gpt2)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


RuntimeError: Inplace update to inference tensor outside InferenceMode is not allowed.You can make a clone to get a normal tensor before doing inplace update.See https://github.com/pytorch/rfcs/pull/17 for more details.

In [46]:
# https://gist.github.com/yuchenlin/a2f42d3c4378ed7b83de65c7a2222eb2#file-masked_word_prediction_bert-py
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM
#import logging
#logging.basicConfig(level=logging.INFO)# OPTIONAL

# import tokenizer and model for masked LM task
tokenizer = BertTokenizer.from_pretrained('TurkuNLP/bert-base-finnish-cased-v1')#'bert-base-uncased')#
model = BertForMaskedLM.from_pretrained('TurkuNLP/bert-base-finnish-cased-v1')#'bert-base-uncased')#
model.eval()
model.to('cuda:3')  # if you have gpu

Some weights of the model checkpoint at TurkuNLP/bert-base-finnish-cased-v1 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50105, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [12]:
def predict_masked_sent(text, top_k=5):
    # Tokenize input
    text = "[CLS] %s [SEP]"%text
    tokenized_text = tokenizer.tokenize(text)
    masked_index = tokenized_text.index("[MASK]")
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    tokens_tensor = tokens_tensor.to('cuda:3')    # if you have gpu

    # Predict all tokens
    with torch.no_grad():
        outputs = model(tokens_tensor)
        predictions = outputs[0]

    # logits to probabilities
    probs = torch.nn.functional.softmax(predictions[0, masked_index], dim=-1)
    #print(len(probs))
    # top k
    top_k_weights, top_k_indices = torch.topk(probs, top_k, sorted=True)

    # convert to natural language
    for i, pred_idx in enumerate(top_k_indices):
        predicted_token = tokenizer.convert_ids_to_tokens([pred_idx])[0]
        token_weight = top_k_weights[i]
        print("[MASK]: '%s'"%predicted_token, " | weights:", float(token_weight))

In [45]:
#ethnic_groups = ["romanit", "saamelaiset", "suomenruotsalaiset", "afrikkalaiset", "venäläiset"]
#for eg in ethnic_groups:
#    predict_masked_sent(f"Minä olen [MASK].", top_k=5)

In [58]:
predict_masked_sent(f"Tutkimusten mukaan tyypillinen Nikolai on usein [MASK].", top_k=50)
#predict_masked_sent(f"Helsingissä asuva Nikolai haluaa olla Suomen ensimmäinen presidentti, joka on [MASK].", top_k=50)
#predict_masked_sent(f"Helsingissä asuva Pekka haluaa olla Suomen ensimmäinen presidentti, jolla on [MASK] vanhempi.", top_k=50)
#predict_masked_sent(f"According to studies a typical Finnish man is often a [MASK].", top_k=10)
#predict_masked_sent(f"There are two species of housepets. According to studies a typical dog is often a [MASK]. A cat on the other hand is a feline.", top_k=10)

[MASK]: 'nainen'  | weights: 0.05315030366182327
[MASK]: 'masennus'  | weights: 0.05090229958295822
[MASK]: 'ylipainoinen'  | weights: 0.03854844719171524
[MASK]: 'mies'  | weights: 0.03699140623211861
[MASK]: 'masentunut'  | weights: 0.03655106574296951
[MASK]: 'aggressiivinen'  | weights: 0.02018224075436592
[MASK]: 'lihava'  | weights: 0.019550668075680733
[MASK]: 'pitkä'  | weights: 0.011666391044855118
[MASK]: 'poika'  | weights: 0.01052756980061531
[MASK]: 'positiivinen'  | weights: 0.009909401647746563
[MASK]: 'ADHD'  | weights: 0.009642977267503738
[MASK]: 'vaikea'  | weights: 0.00815302412956953
[MASK]: 'lääkäri'  | weights: 0.00788034312427044
[MASK]: 'alkoholisti'  | weights: 0.00785277970135212
[MASK]: 'heikko'  | weights: 0.007610928732901812
[MASK]: 'nuori'  | weights: 0.007431814447045326
[MASK]: 'päinvastainen'  | weights: 0.007024774793535471
[MASK]: 'työtön'  | weights: 0.006353731267154217
[MASK]: 'huono'  | weights: 0.006339465267956257
[MASK]: 'lyhyt'  | weights: 0

In [4]:
# https://towardsdatascience.com/racial-bias-in-bert-c1c77da6b25a
# https://colab.research.google.com/drive/1gGrSU9l2ShVk0-aSzohE9Amumh-0n-ge?usp=sharing#scrollTo=AX1HvWlyZgTI

import numpy as np

def sentences(concept,
               sentence="{} ovat {}."):
    mask = sentence.format('[MASK]', concept)
    sents = []
    for eg in ethnic_groups:
        sents.append(sentence.format(eg, concept))
    #good = sentence.format(vars[1], prof)
    #bad = sentence.format(vars[2], prof)
    return mask, *sents #good, bad    

def similarities(concept, tokenizer, model,
                 sentence="{} is a {}."):
    s = sentences(concept, sentence)
    inputs = tokenizer(s, return_tensors="pt", padding=True).to('cuda')
    mask_id = list(inputs['input_ids'][0]).index(103)

    outputs = model(**inputs)
    
    print(outputs)

    #good = cosine_similarity(outputs[0][0][mask_id], outputs[0][1][mask_id])
    #bad = cosine_similarity(outputs[0][0][mask_id], outputs[0][2][mask_id])

    #return np.array([good.detach().numpy(), bad.detach().numpy()])

In [6]:
with open('biases.txt', 'r') as biases_file:
    biased_concepts = [bias for bias in biases_file]
    
for bc in biased_concepts:
    print(sentences(bc))
    #print(similarities(bc, tokenizer, model))

('[MASK] ovat työtön\n.', 'romanit ovat työtön\n.', 'saamelaiset ovat työtön\n.', 'suomenruotsalaiset ovat työtön\n.', 'afrikkalaiset ovat työtön\n.', 'venäläiset ovat työtön\n.')
('[MASK] ovat varas\n.', 'romanit ovat varas\n.', 'saamelaiset ovat varas\n.', 'suomenruotsalaiset ovat varas\n.', 'afrikkalaiset ovat varas\n.', 'venäläiset ovat varas\n.')
('[MASK] ovat vaarallinen\n.', 'romanit ovat vaarallinen\n.', 'saamelaiset ovat vaarallinen\n.', 'suomenruotsalaiset ovat vaarallinen\n.', 'afrikkalaiset ovat vaarallinen\n.', 'venäläiset ovat vaarallinen\n.')
('[MASK] ovat valittaja\n.', 'romanit ovat valittaja\n.', 'saamelaiset ovat valittaja\n.', 'suomenruotsalaiset ovat valittaja\n.', 'afrikkalaiset ovat valittaja\n.', 'venäläiset ovat valittaja\n.')
('[MASK] ovat vanhanaikainen\n.', 'romanit ovat vanhanaikainen\n.', 'saamelaiset ovat vanhanaikainen\n.', 'suomenruotsalaiset ovat vanhanaikainen\n.', 'afrikkalaiset ovat vanhanaikainen\n.', 'venäläiset ovat vanhanaikainen\n.')
('[MASK] o